#  Introduction
##  About this dataset

For each dataset, the fields are:
* Time
* Ls
* LT
* Tsurf: surface temperature (in Kelvin)
* Psurf: surface pressure (in Pascals)
* CO2ice: Surface carbon dioxide ice (in kg per metre squared)
* cloud: water ice column (in opacity)
* vapour: water vapour column (in kg per metre squared)
* u_wind: Zonal wind (west-east) (metres per second)
* v_wind: Meridional wind (north-south) (metres per second)
* dust: dust column (in opacity)
* temp: atmospheric temperature at a height of abour 2.5 km (in Kelvin)

##  About this notebook
This notebook primarily deals with regression analysis of the dataset.


__Limitations__
No optimisation for memory, runtime, or readability.

# 2. Preprocessing the data

## 2.1. Defining Features and Labels
Machine learning algorithms operate on _features_ to predict _labels_.

* A __feature__ is an attribute of the system that affects the output.
Features act as "inputs" to the model.
Ideally, features are _independent_ variables.
* A __label__ is the value being predicted.
Labels act as "outputs" of the model.

### 2.1.1. Features
At every timestamp within each day, there are values for all other variables.
No other variables impact the values of time or date.
Therefore __date__ and __time of day__ are _independent_ variables.

Temperature, pressure, and humidity do not directly affect one another significantly, but since they are all properties which describe the local atmosphere, they do not vary independently from one another.
Similarly, all three of these variables have a stong relationship to time of day.

CO2ice is always 0.

Therefore we consider the following variables to be _features_ to the machine learning algorithm:
* Tsurf: surface temperature (in Kelvin)
* cloud: water ice column (in opacity)
* vapour: water vapour column (in kg per metre squared)
* u_wind: Zonal wind (west-east) (metres per second)
* v_wind: Meridional wind (north-south) (metres per second)
* dust: dust column (in opacity)
* temp: atmospheric temperature at a height of abour 2.5 km (in Kelvin)

Further exploration of the dataset may modify this list, but for now this is our best guess.

### 2.1.2. Labels
The goal is to model surface pressure (Psurf) based on the available features.

## Import/install useful libraries

In [67]:
!pip install lux-api==0.5.1
!pip install statsmodels
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, median_absolute_error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 KB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.3/478.3 KB 6.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 KB 7.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.4 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 KB 1.3 MB/s eta 0:00:00
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)


  Created wheel for lux-api: filename=lux_api-0.5.1-py3-none-any.whl size=185620 sha256=997db99756d157c707104a299b11a38bcacf621625af2a75f1e48d406c9281b3
  Stored in directory: /Users/suprety/Library/Caches/pip/wheels/58/c7/50/1cc05b708e6be379f40d0609f2878bbd89c947021b9a2c6085
  Created wheel for lux-widget: filename=lux_widget-0.1.11-py3-none-any.whl size=2540460 sha256=27cf63d3ed51daf3f70360d1637865ce911658123698a9bf98cb4c72738c0e7c
  Stored in directory: /Users/suprety/Library/Caches/pip/wheels/e5/b4/32/50f87f9ee0473d4c3278a2b8ec8ea08f49d9ac08534fdb9efd
  Created wheel for psutil: filename=psutil-5.9.0-cp38-cp38-macosx_10_14_arm64.whl size=264786 sha256=dcbf11dab2be5d8139fa67d4a6b2cfdc73cd4e42f4e721c724cfb829452f44cd
  Stored in directory: /Users/suprety/Library/Caches/pip/wheels/9c/28/d8/45cbe6aa547b8b581dfa337cb3aaec76f3029d22f1199fff9e
Successfully built lux-api lux-widget psutil


## Begin with code snippets

In [24]:
#Import data

df= pd.read_csv('data/insight_openmars_training_time.csv')
df.drop(['Ls','LT','CO2ice'],axis=1,inplace=True)
target = df['Psurf']
df.drop(labels=['Psurf'], axis=1,inplace = True)
df.insert(1, 'Psurf', target)

df['Time']= pd.to_datetime(df['Time'])

print(df.head())

df.describe()

df.info()

                 Time    Psurf    Tsurf  cloud  vapour  u_wind  v_wind   dust  \
0 1998-07-15 21:23:39  721.113  264.042  0.092   0.027  -7.451   8.604  0.428   
1 1998-07-15 23:26:53  705.090  274.736  0.145   0.026  -7.053   4.934  0.427   
2 1998-07-16 01:30:07  700.691  265.939  0.105   0.026  -6.825  -0.063  0.427   
3 1998-07-16 03:33:21  697.252  238.624  0.134   0.025  -5.373  -4.048  0.426   
4 1998-07-16 05:36:35  717.146  213.634  0.139   0.026  -3.899  -3.133  0.426   

      temp  
0  179.686  
1  174.502  
2  173.429  
3  173.556  
4  174.789  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72196 entries, 0 to 72195
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Time    72196 non-null  datetime64[ns]
 1   Psurf   72196 non-null  float64       
 2   Tsurf   72196 non-null  float64       
 3   cloud   72196 non-null  float64       
 4   vapour  72196 non-null  float64       
 5   u_wind  72196 

In [47]:
# Dividing data into features and predictor variables
predictors = ['Tsurf',  'cloud',  'vapour',  'u_wind',  'v_wind',   'dust', 'temp']
df2 = df[['Psurf'] + predictors]
df2.head()

,Psurf,Tsurf,cloud,vapour,u_wind,v_wind,dust,temp
0,721.113,264.042,0.092,0.027,-7.451,8.604,0.428,179.686
1,705.090,274.736,0.145,0.026,-7.053,4.934,0.427,174.502
2,700.691,265.939,0.105,0.026,-6.825,-0.063,0.427,173.429
3,697.252,238.624,0.134,0.025,-5.373,-4.048,0.426,173.556
4,717.146,213.634,0.139,0.026,-3.899,-3.133,0.426,174.789


In [48]:
# separate our my predictor variables (X) from my outcome variable y
X = df2[predictors]
y = df2['Psurf']

# Add a constant to the predictor variable set to represent the Bo intercept
X = sm.add_constant(X)
X.iloc[:5, :5]

,const,Tsurf,cloud,vapour,u_wind
0,1.0,264.042,0.092,0.027,-7.451
1,1.0,274.736,0.145,0.026,-7.053
2,1.0,265.939,0.105,0.026,-6.825
3,1.0,238.624,0.134,0.025,-5.373
4,1.0,213.634,0.139,0.026,-3.899


In [49]:
# (1) select a significance value
alpha = 0.05

# (2) Fit the model
model = sm.OLS(y, X).fit()

# (3) evaluate the coefficients' p-values
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Psurf   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     1911.
Date:                Thu, 24 Feb 2022   Prob (F-statistic):               0.00
Time:                        13:20:55   Log-Likelihood:            -3.7578e+05
No. Observations:               72196   AIC:                         7.516e+05
Df Residuals:                   72188   BIC:                         7.517e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        816.4173      3.095    263.772      0.000     810.351     822.484
Tsurf         -0.1693      0.006    -27.282      0.000      -0.181      -0.157
cloud        -15.6108      2.986     -5.227      0.000     -21.464      -9.758
vapour     -1227.0244     36.202    -33.894      0.000   -1297.979   -1156.069
u_wind         3.5723      0.053     66.968      0.000       3.468       3.677
v_wind        -0.5472      0.037    -14.761      0.000      -0.620      -0.475
dust           8.8425      0.474     18.644      0.000       7.913       9.772
temp          -0.1692      0.017    -10.190      0.000      -0.202      -0.137
==============================================================================
Omnibus:                     4710.039   Durbin-Watson:                   0.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3629.105
Skew:                          -0.458   Prob(JB):                         0.00
Kurtosis:                       2.394   Cond. No.                     6.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [50]:
# first remove the const column because unlike statsmodels, SciKit-Learn will add that in for us
X = X.drop('const', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)


In [51]:
# instantiate the regressor class
regressor = LinearRegression()

# fit the build the model by fitting the regressor to the training data
regressor.fit(X_train, y_train)

# make a prediction set using the test set
prediction = regressor.predict(X_test)

# Evaluate the prediction accuracy of the model
from sklearn.metrics import mean_absolute_error, median_absolute_error
print("The Explained Variance: %.2f" % regressor.score(X_test, y_test))
print("The Mean Absolute Error: %.2f" % mean_absolute_error(y_test, prediction))
print("The Median Absolute Error: %.2f" % median_absolute_error(y_test, prediction))


The Explained Variance: 0.17
The Mean Absolute Error: 36.35
The Median Absolute Error: 33.70


## Above values cannot be compared to those obtained in pressure_prediction notebook as the feature values are not normalised/scaled between -1 and 1. 

## We will do the normalisation and perform regression again.

In [62]:
# Use normalisation as used in pressure_prediction notebook and then perform regression again to compare with lstm model
scalers={}
for i in df2.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(df2[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    df2[i]=s_s


/var/folders/n8/4f8t1lrj2zz6hv02gtnmv46m0000gr/T/ipykernel_17525/2308460018.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i]=s_s
/var/folders/n8/4f8t1lrj2zz6hv02gtnmv46m0000gr/T/ipykernel_17525/2308460018.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i]=s_s
/var/folders/n8/4f8t1lrj2zz6hv02gtnmv46m0000gr/T/ipykernel_17525/2308460018.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [63]:
#Scaled dataframe
df2

,Psurf,Tsurf,cloud,vapour,u_wind,v_wind,dust,temp
0,0.028370,0.532421,-0.786790,0.388889,-0.379457,0.529415,-0.839490,-0.275933
1,-0.100824,0.725170,-0.663963,0.333333,-0.357197,0.335343,-0.839865,-0.381352
2,-0.136294,0.566612,-0.756663,0.333333,-0.344445,0.071098,-0.839865,-0.403172
3,-0.164023,0.074286,-0.689455,0.277778,-0.263234,-0.139631,-0.840240,-0.400590
4,-0.003616,-0.376134,-0.677868,0.333333,-0.180794,-0.091246,-0.840240,-0.375516
...,...,...,...,...,...,...,...,...
72191,0.104042,0.009940,-0.955968,-0.333333,0.141755,0.530948,-0.854491,-0.035730
72192,0.106098,0.548769,-0.965238,-0.333333,-0.349982,0.422913,-0.854491,-0.115180
72193,-0.109718,0.743915,-0.972190,-0.333333,-0.224531,0.218635,-0.854491,-0.164372
72194,-0.175440,0.584762,-0.972190,-0.333333,-0.146005,-0.215568,-0.854491,-0.183813


In [66]:
X = df2[predictors]
y = df2['Psurf']
# X = X.drop('const', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

regressor = LinearRegression()

# fit the build the model by fitting the regressor to the training data
regressor.fit(X_train, y_train)

# make a prediction set using the test set
prediction = regressor.predict(X_test)

# Evaluate the prediction accuracy of the model
print("The Explained Variance: %.2f" % regressor.score(X_test, y_test))
print("The Mean Absolute Error: %.2f" % mean_absolute_error(y_test, prediction))
print("The Median Absolute Error: %.2f" % median_absolute_error(y_test, prediction))

The Explained Variance: 0.17
The Mean Absolute Error: 0.29
The Median Absolute Error: 0.27


# As we can see, the mean absolute error (mae) above is 0.29 with linear regression. The mae obtained in the lstm model in pressure_prediction notebook is around 0.123.